Aluno: Vinícius Santos Monteiro
Num USP: 11932463

Sobre a atividade:

    * Seta [frente] e [trás] acelera e freia/(dá ré) na nave respectivamente
    * [Espaço] freia totalmente a nave
    * [ESC] finaliza a janela
    * [+] e [-] alteram a escala da nave respectivamente
    * Quando a nave atravessa ultrapassa o limite -1 ou +1 de qualquer eixo ele aparece no lado oposto

In [25]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

In [26]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(900, 900, "Atividade 1", None, None)
glfw.make_context_current(window)

In [27]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat;
        void main(){
            gl_Position = mat * vec4(position,0.0,1.0);
        }
        """

fragment_code = """
        void main(){
            gl_FragColor = vec4(0.0, 0.0, 0.0, 1.0);
        }
        """

In [28]:

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

In [29]:

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [30]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)


In [31]:
# preparando espaço para 4 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(70, [("position", np.float32, 2)])

# preenchendo as coordenadas de cada vértice
vertices['position'] = [
                            (0, -0.12),
                            (0.04, -0.06),
                            (0.05, -0.05),
                            (0.06, -0.05),
                            (0.07, -0.06),
                            (0.08, -0.08),
                            (0.09, -0.06),
                            (0.1, -0.05),
                            (0.11, -0.05),
                            (0.12, -0.06),
                            (0.13, -0.08),
                            (0.13, -0.09),
                            (0.12, -0.11),
                            (0.11, -0.12),
                            (0.14, -0.11),
                            (0.17, -0.09),
                            (0.19, -0.07),
                            (0.2, -0.05),
                            (0.21, -0.02),
                            (0.21, 0.01),
                            (0.2, 0.04),
                            (0.19, 0.06),
                            (0.17, 0.08),
                            (0.14, 0.1),
                            (0.11, 0.11),
                            (0.12, 0.1),
                            (0.13, 0.08),
                            (0.13, 0.07),
                            (0.12, 0.05),
                            (0.11, 0.04),
                            (0.09, 0.03),
                            (0.06, 0.03),
                            (0.03, 0.04),
                            (0.03, 0.12),
                            (0.01, 0.09),
                            (-0.01, 0.09),
                            (-0.03, 0.12),
                            (-0.03, 0.04),
                            (-0.06, 0.03),
                            (-0.09, 0.03),
                            (-0.11, 0.04),
                            (-0.12, 0.05),
                            (-0.13, 0.07),
                            (-0.13, 0.08),
                            (-0.12, 0.1),
                            (-0.11, 0.11),
                            (-0.14, 0.1),
                            (-0.17, 0.08),
                            (-0.19, 0.06),
                            (-0.2, 0.04),
                            (-0.21, 0.01),
                            (-0.21, -0.02),
                            (-0.20, -0.05),
                            (-0.19, -0.07),
                            (-0.17, -0.09),
                            (-0.14, -0.11),
                            (-0.11, -0.12),
                            (-0.12, -0.11),
                            (-0.13, -0.09),
                            (-0.13, -0.08),
                            (-0.12, -0.06),
                            (-0.11, -0.05),
                            (-0.1, -0.05),
                            (-0.09, -0.06),
                            (-0.08, -0.08),
                            (-0.07, -0.06),
                            (-0.06, -0.05),
                            (-0.05, -0.05),
                            (-0.04, -0.06),
                            (0, -0.12)
                        ]


In [32]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [33]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

In [34]:
# coeficiente de escala
s_inc = 1.0

should_close = False
acceleration = 0.0

def key_event(window,key,scancode,action,mods):
    global s_inc, should_close, acceleration
    
    # ESC para fechar a janela
    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        print('should close')
        should_close = True

    if action == glfw.PRESS:
        if key == glfw.KEY_UP:
            acceleration += 0.001
        if key == glfw.KEY_DOWN:
            acceleration -= 0.001
        if key == glfw.KEY_SPACE:
            acceleration = 0.0

    if key == glfw.KEY_KP_ADD: s_inc += 0.05

    if key == glfw.KEY_KP_SUBTRACT: s_inc -= 0.05
        
    print(key)
    
glfw.set_key_callback(window,key_event)


# Função para converter coordenadas da janela para o OpenGL
def normalize_coordinates(x, y, width, height):
    normalized_x = (2 * x / width) - 1
    normalized_y = 1 - (2 * y / height)
    return normalized_x, normalized_y

def window_to_opengl_coordinates(x, y, width, height):
    normalized_x, normalized_y = normalize_coordinates(x, y, width, height)
    opengl_x = normalized_x
    opengl_y = normalized_y
    return opengl_x, opengl_y


def mouse_position_event(window, xpos, ypos):
    global mouse_x, mouse_y
    
    # print('[mouse position event] xpos=', xpos)
    # print('[mouse position event] ypos=', ypos)
    # print('-------')
    mouse_x, mouse_y = window_to_opengl_coordinates(xpos, ypos, 900, 900)

glfw.set_cursor_pos_callback(window, mouse_position_event)


In [35]:
glfw.show_window(window)

In [36]:
angulo = 0.0
s_x = 1.0
s_y = 1.0

mouse_x = 0.0
mouse_y = 0.0

x_inc = 0.0
y_inc = 0.0
t_x = 0.0
t_y = 0.0

r_inc = 0.0

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window) and not should_close:

    # Limpeza do buffer
    glClear(GL_COLOR_BUFFER_BIT)
    
    t_x += x_inc
    t_y += y_inc
    angulo += r_inc
    
    s_x = s_inc
    s_y = s_inc
    
    c = math.cos( math.radians(angulo) )
    s = math.sin( math.radians(angulo) )

    # Calcula o ângulo entre a posição do mouse e a posição do objeto
    delta_x = mouse_x - t_x
    delta_y = mouse_y - t_y
    angulo = math.degrees(math.atan2(delta_y, delta_x)) - 90
    
    glfw.poll_events() 

    
    # glPolygonMode(GL_FRONT_AND_BACK,GL_LINE) ## ative esse comando para enxergar os triângulos
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)

    mouse_x, mouse_y = glfw.get_cursor_pos(window)
    mouse_x, mouse_y = window_to_opengl_coordinates(mouse_x, mouse_y, 900, 900)
    direction_x = mouse_x - t_x
    direction_y = mouse_y - t_y
    length = math.sqrt(direction_x ** 2 + direction_y ** 2)
    if length != 0:
        direction_x /= length
        direction_y /= length
    t_x += direction_x * acceleration
    t_y += direction_y * acceleration

    # Limites da tela
    screen_width = 2
    screen_height = 2

    # Verificar se o objeto ultrapassou os limites da tela
    if t_x > 1:
        t_x -= screen_width
    elif t_x < -1:
        t_x += screen_width
    if t_y > 1:
        t_y -= screen_height
    elif t_y < -1:
        t_y += screen_height

    #Draw Batman Logo
    
    mat_rotation = np.array([  c  , -s , 0.0, 0.0, 
                               s  , c  , 0.0, 0.0, 
                               0.0, 0.0, 1.0, 0.0, 
                               0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_scale =    np.array([  s_x  , 0.0 , 0.0, 0.0, 
                               0.0  , s_y  , 0.0, 0.0, 
                               0.0, 0.0, 1.0, 0.0, 
                               0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_translation = np.array([  1.0, 0.0, 0.0, t_x, 
                                  0.0, 1.0, 0.0, t_y, 
                                  0.0, 0.0, 1.0, 0.0, 
                                  0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_translation,mat_rotation)
    mat_transform = multiplica_matriz(mat_transform,mat_scale)

    loc = glGetUniformLocation(program, "mat")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)


    glDrawArrays(GL_LINE_LOOP, 0, 70)

    

    glfw.swap_buffers(window)

glfw.terminate()

265
265
265
265
265
265
265
265
265
265
265
265
265
265
265
265
334
334
334
334
333
333
333
333
333
333
333
333
333
333
333
333
333
333
333
333
333
333
333
333
264
264
32
32
264
264
264
264
264
264
264
264
264
264
264
264
264
264
264
264
264
264
264
264
265
265
265
265
should close
256
